|평가문항	|상세기준|
|:-|:-|
|1. 그림에 맞게 딥러닝 모델을 잘 설계하였는가?	|그림에 맞게 딥러닝 모델을 설계했다.|
|2. 잘못 들어간 test label들을 수정하였는가?	|틀린 Label 22개를 전부 시각화했으며 Label 또한 수정했다.|
|3. KerasTuner로 하이퍼파라미터 튜닝을 진행하였는가?	|KerasTuner를 정상작동하였고 하이퍼파라미터 튜닝을 제대로 수행했다. 또한 최적화된 하이퍼파라미터로 모델 학습을 진행했다.|

In [1]:
# !mkdir ~/aiffel/mlops
# !pip install keras-tuner

In [2]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [3]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
# CIFAR-10 데이터셋을 로드합니다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR-10 데이터셋을 로드합니다.
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 라벨이 잘못된 데이터의 ID와 정정된 라벨 인덱스입니다.
# CIFAR-10 라벨: airplane : 0, automobile : 1, bird : 2, cat : 3, deer : 4, dog : 5, frog : 6, horse : 7, ship : 8, truck : 9
label_corrections = {
    2405: 6, # frog
    6877: 8, # ship
    8058: 7, # horse
    # 2532: -1, # Neither airplane nor automobile, -1 if no consensus
    7657: 7, # horse
    # 1969: -1, # Neither automobile nor truck
    2804: 5, # dog
    # 6792: -1, # Neither cat nor truck
    1227: 5, # dog
    5191: 5, # dog
    5690: 4, # deer
    1718: 8, # ship
    2592: 4, # deer
    4794: 2, # bird
    5960: 3, # cat
    # 165: -1, # Neither deer nor bird
    9227: 9, # truck
    5632: 5, # dog
    9352: 9, # truck
    7846: 3, # cat
    6966: 8, # ship
    5468: 3, # cat
}

# 잘못된 라벨을 수정합니다.
for img_id, correct_label in label_corrections.items():
    y_test[img_id] = correct_label

# 수정된 라벨을 검증합니다 (처음 몇 개만).
for img_id, correct_label in list(label_corrections.items())[:5]:
    print(f"ID: {img_id}, Corrected Label: {correct_label}")

ID: 2405, Corrected Label: 6
ID: 6877, Corrected Label: 8
ID: 8058, Corrected Label: 7
ID: 7657, Corrected Label: 7
ID: 2804, Corrected Label: 5


In [4]:
# 데이터 타입을 float32로 변환하고 [0,1] 범위로 정규화합니다.
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 훈련 데이터를 훈련 세트와 검증 세트로 나눕니다.
X_train, X_val, y_train_raw, y_val_raw = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# 원-핫 인코딩으로 변환합니다.
y_train = tf.keras.utils.to_categorical(y_train_raw, 10)
y_val = tf.keras.utils.to_categorical(y_val_raw, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [5]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [6]:
import tensorflow as tf
import keras_tuner as kt

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('filters_1', min_value=32, max_value=64, step=32),
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Int('filters_2', min_value=64, max_value=128, step=64),
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('units', min_value=256, max_value=512, step=256),
        activation='relu'
    ))
    model.add(tf.keras.layers.Dropout(
        rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    ))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [7]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

# 해당 모델 학습시간은 약 10분정도 걸립니다!
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 28s]
accuracy: 0.673799991607666

Best accuracy So Far: 0.6833000183105469
Total elapsed time: 00h 05m 48s


In [8]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
average_pooling2d_22 (Averag (None, 15, 15, 32)        0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 15, 15, 32)        128       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
average_pooling2d_23 (Averag (None, 6, 6, 64)          0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 6, 6, 64)          256       
_________________________________________________________________
flatten_11 (Flatten)         (None, 2304)            

In [9]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1250/1250 [==============================] - 5s 3ms/step - loss: 1.4972 - accuracy: 0.5116
Epoch 2/5
1250/1250 [==============================] - 4s 3ms/step - loss: 0.9851 - accuracy: 0.6583
Epoch 3/5
1250/1250 [==============================] - 4s 4ms/step - loss: 0.7890 - accuracy: 0.7253
Epoch 4/5
1250/1250 [==============================] - 4s 3ms/step - loss: 0.6376 - accuracy: 0.7772
Epoch 5/5
1250/1250 [==============================] - 4s 3ms/step - loss: 0.4968 - accuracy: 0.8278


In [11]:
model.evaluate(X_val, y_val)

313/313 [==============================] - 1s 3ms/step - loss: 1.0232 - accuracy: 0.6798


[1.023191213607788, 0.6797999739646912]

모델은 val 데이터셋에 대해 약 67.98%의 정확도로 1.023의 손실을 가지고 있다...

In [12]:
save_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1'
fname = os.path.join(save_path, 'model')
model.save(fname)

INFO:tensorflow:Assets written to: /aiffel/aiffel/mlops/best_model/1/model/assets


# 회고
밀려서 부랴부랴 끝내기 급급했네요...! 시간이 남으면 추가 미션까지 진행해보겠습니다.